## 4. Primal Heuristic方法

本notebook主要包括B&B方法中一部分常见的primal heuristic方法（rounding + diving）的实现。

首先是准备工作：导入相关的计算工具包、从文件util_lec_4.py中读取上一个notebook中完成的内容。

In [1]:
import time
import math
import numpy as np
from cylp.cy import CyClpSimplex
from util import *
from util_lec_4 import *
np.set_printoptions(suppress=True,precision=4)

下面的代码同样包括两部分：

* 对原有**Node类**和**BBSolver类**的扩展
* 在**HeuristicManager类**中实现基于rounding和diving的primal heuristic

Node类的扩展主要是在update方法中调用多种heuristic方法来寻找可行解。

In [2]:
class NodeExtend(Node):
    def update(self,sol,obj,solver):
        super().update(sol,obj,solver)
        ## 只有在LP子问题可行且当前解整数不可行时，通过heuristic寻找可行解才有意义
        if obj < MAX and not self.sol_feas:
            hm = solver.heuristic_manager
            ## 遍历调用多种heuristic
            for heuristic in [hm.simple_rounding,hm.int_rounding,hm.diving]:
                feas_sol_flag,feas_sol,feas_obj = heuristic(solver,self)
                if feas_sol_flag:
                    solver.update_global_sol(feas_obj,feas_sol)
                    break

BBSolver类的扩展则主要包括对HeuristicManager的调用以及对新可行解的处理。

In [3]:
class BBSolverExtend(BBSolver):
    def __init__(self,problem,branch_type=None,select_type=None,propagate_type=None):
        ## 进行基础的初始化
        super().__init__(problem,branch_type,select_type,propagate_type)
        ## 初始化heuristic manager
        self.heuristic_manager = HeuristicManager(self.problem,self)
        
    def update_global_sol(self,obj,sol):
        ## 更新全局最优解
        if obj < self.global_obj:
            self.global_obj,self.global_sol = obj,sol.copy()
        
    def reset_problem_bounds(self):
        ## 恢复变量上下界
        self.vars_ub[:] = self.init_vars_ub
        self.vars_lb[:] = self.init_vars_lb
        
    def create_node(self,parent_node,var_idx,var_lb,var_ub,var_sol,var_bnd):
        '''
        相比原来版本，只是将Node改为Node Extend
        '''
        new_node = NodeExtend(node_idx=self.next_node_idx,parent=parent_node,cost_manager=self.cost_manager,
                        var_idx=var_idx,var_lb=var_lb,var_ub=var_ub,var_sol=var_sol,var_bnd=var_bnd,
                        init_var_lb=self.init_vars_lb[var_idx],init_var_ub=self.init_vars_ub[var_idx])
        parent_node.childs += [new_node]
        self.add_node(new_node)
        
    def solve_root_node(self):
        '''
        相比原来版本，只是将Node改为Node Extend
        '''
        ## 生成根结点
        root_node = NodeExtend()
        self.root_node = root_node
        self.add_node(root_node)
        ## 执行简化版的process
        self.unprocessed_node_idxs.remove(root_node.idx)
        root_node.activate(self) ## 激活node，生成局部信息
        self.node_solve(root_node) ## 求解子问题
        self.root_sol = root_node.sol ## 根结点（初始问题）的LP relaxation最优解
        self.global_LB = root_node.LB ## 更新BB树的上下界信息
        status = SolveStatus.ONGOING
        if not root_node.sol_feas and self.global_LB < self.global_obj:
            self.branch(root_node)
        else:
            status = SolveStatus.OPT
        root_node.deactivate() ## 释放内存
        return status

在完成了上述准备后，下面，我们开始实现primal heuristics。在这个notebook中，我们主要实现以下几种方法：

* rounding：给定一个LP子问题和相应的（整数不可行）解，尝试通过对整数不可行变量做rounding来寻找可行解。下面的代码实现了以下两种方式：
    * simple rounding: 如果对每个不可行整数变量，都存在某个rounding方向不受到LP问题的线性约束限制，则可以直接rounding获取可行解。具体来说，对于变量$x_j$和约束$b_i^- \leq a_i^T x \leq b_i^+$，如果$a_{ij} > 0$且$b_i^+ = \infty$、或者$a_{ij} < 0$且$b_i^- = \infty$，那么该约束对变量$x_j$的向上方向没有限制作用。我们称对$x_j$的向上/下方向有限制作用的约束个数为locks；那么simple rounding要求对每个整数不可行变量，都存在某个rounding方向的locks数为0。
    * integer rounding: 
        1. 首先，放松连续变量对各线性约束的作用：通过propagator计算连续变量对应的约束隐含上下界$b_{im}^-,b_{im}^+$，然后计算约束上下界$b^-,b^+$与其的差$b_I^-,b_I^+$，作为整数变量单独的线性约束上下界。
        2. 其次，尝试通过rounding找到一个满足放松约束后的整数可行解。具体来说，我们先在考虑locks数和rounding后不可行程度的情况下，尝试对整数不可行变量做rounding；如果造成了约束不可行性，那么进一步寻找其他整数不可行变量缓解此约束不可行的程度。如果找不到这样的整数不可行变量，那么此次对可行解的search没有成功，直接abort。
        3. 最后，如果找到了一个满足放松约束后的整数可行解，那么进一步在原问题中固定各整数变量的值为该解，然后求解子LP问题；如果LP问题可行，那么解必然是原问题的一个可行解。否则，此次对可行解的search没有成功，直接abort。
* diving：给定一个LP子问题和相应的（整数不可行）解，在diving的每一步中，先选取一个整数不可行的变量，然后向上/下其中一个方向加强变量上下界，然后重新求解LP问题，直到获得一个整数可行解，或是LP问题无解为止。这种做法类似于简化版的DFS+branching，因此虽然每一步都需要求解一个LP问题，但是计算效率不差。在选取加强上下界的变量和方向时，下面的代码实现了以下两种方式：
    * infeasibility diving: 选取整数不可行程度最大的变量，向上/下整数中距离更近的一侧加强界。
    * coefficient diving: 选取上/下locks数最小的变量，向locks数小的一侧加强界；如果有多个变量有一样的locks数，则对这部分变量进行infeasibility diving的操作。

具体的实现逻辑可以参考下面代码中的注释。

In [4]:
@unique
class HeuristicDiveType(Enum):
    ## diving方法类型
    INF = 1 ## 根据不可行程度选择dive变量
    COEF = 2 ## 根据locks数选择dive变量

import math
import random
    
class HeuristicManager:
    def __init__(self,problem,solver):
        self.problem = problem
        self.dive_type = HeuristicDiveType.COEF ## diving的具体方法

        ## 初始化辅助变量
        ## 提前提取问题的相关信息，节省后续调用开销
        self.primal_sol = solver.primal_sol
        self.A_csc = problem.coefMatrix.tocsc()
        self.A_csr = self.A_csc.tocsr()
        self.obj_coef = problem.objective.copy()
        self.int_vars_bool = solver.int_vars_bool
        self.vars_lb,self.vars_ub = solver.vars_lb,solver.vars_ub
        self.init_vars_lb,self.init_vars_ub = solver.init_vars_lb,solver.init_vars_ub
        self.rows_lb,self.rows_ub = problem.constraintsLower.copy(),problem.constraintsUpper.copy()
        self.num_row,self.num_col = len(self.rows_lb),len(self.vars_lb)
        self.rows_range,self.cols_range = \
            np.arange(self.num_row,dtype=np.int32),np.arange(self.num_col,dtype=np.int32)
        
        ## 计算locks：b^- <= Ax <= b^+
        ## downward lock if a > 0 and b^- > -INF, or a < 0 and b^+ < INF
        ## upward lock if a < 0 and b^- > -INF, or a > 0 and b^+ < INF
        AT_posi,AT_nega = (self.A_csc > 0).astype(int).T,(self.A_csc < 0).astype(int).T
        
        rows_lb_valid,rows_ub_valid = self.rows_lb > -INF,self.rows_ub < INF
        self.down_locks = AT_posi.dot(rows_lb_valid) + AT_nega.dot(rows_ub_valid)
        self.up_locks = AT_nega.dot(rows_lb_valid) + AT_posi.dot(rows_ub_valid)
        self.min_locks = np.minimum(self.down_locks,self.up_locks)
        self.max_locks = np.maximum(self.down_locks,self.up_locks)
        
        ## 计算放松连续变量后的约束上下界和对应整数变量的locks，为int rounding做准备
        As_cont = solver.propagator.matrix_init(self.A_csc[:,~self.int_vars_bool].tocsr())
        var_bnds_cont = solver.propagator.bound_init(self.init_vars_lb[~self.int_vars_bool],
                                                     self.init_vars_ub[~self.int_vars_bool])
        row_im_bnds_cont = solver.propagator.calc_cons_im_bound(As_cont,var_bnds_cont)
        row_bnds = solver.propagator.bound_init(self.rows_lb.copy(),self.rows_ub.copy())
        rows_lb_int,rows_ub_int = np.empty(self.rows_lb.shape),np.empty(self.rows_ub.shape)
        idxs_infeas = (row_bnds[2] > 0) | (row_im_bnds_cont[3] > 0)
        rows_lb_int[idxs_infeas] = -INF
        rows_lb_int[~idxs_infeas] = row_bnds[0][~idxs_infeas] - row_im_bnds_cont[1][~idxs_infeas]
        idxs_infeas = (row_bnds[3] > 0) | (row_im_bnds_cont[2] > 0)
        rows_ub_int[idxs_infeas] = INF
        rows_ub_int[~idxs_infeas] = row_bnds[1][~idxs_infeas] - row_im_bnds_cont[0][~idxs_infeas]
        self.rows_lb_int,self.rows_ub_int = rows_lb_int,rows_ub_int
        rows_lb_int_valid,rows_ub_int_valid = self.rows_lb_int > -INF,self.rows_ub_int < INF
        self.down_locks_int = AT_posi.dot(rows_lb_int_valid) + AT_nega.dot(rows_ub_int_valid)
        self.up_locks_int = AT_nega.dot(rows_lb_int_valid) + AT_posi.dot(rows_ub_int_valid)
        self.min_locks_int = np.minimum(self.down_locks_int,self.up_locks_int)
        self.max_locks_int = np.maximum(self.down_locks_int,self.up_locks_int)
        
        ## 控制各heuristics调用频次的一些超参数
        self.dive_num = 500
        self.diving_count = 0
        self.diving_threshold = 0.02
        
        self.rounding_count = 0
        self.rounding_threshold = 0.1
        
    def check_sol_int_feas(self,sol):
        ## 检查所得解是否整数可行
        delta_infeas = np.abs(sol - np.round(sol))
        sol_bool_infeas = (delta_infeas > TOL) & self.int_vars_bool
        sol_int_infeas = np.where(sol_bool_infeas)[0]
        return sol_int_infeas
        
    def check_sol_feas(self,sol):
        ## 检查所得解是否约束+整数可行
        rows_val = self.A_csr.dot(sol)
        inf_rows,inf_cols = get_inf_idxs_jit(rows_val,self.rows_lb,self.rows_ub,
                                             self.int_vars_bool,sol,self.rows_range,self.cols_range)
        return (len(inf_rows) == 0 and len(inf_cols) == 0)
    
    def _simple_rounding_internal(self,sol,sol_int_infeas):
        '''
        simple rounding: 如果每个整数不可行的变量都有一个方向没有lock，则可以直接round生成可行解
        '''
        if np.any(self.min_locks[sol_int_infeas] > 0):
            return False,None,MAX
        
        feas_sol = sol.copy()
        for idx in sol_int_infeas:
            ## 选择没有locks的方向round
            if self.down_locks[idx] == 0:
                if self.up_locks[idx] == 0:
                    ## 如果两个方向都没有lock，选择对目标函数更有利的方向
                    if self.obj_coef[idx] < 0:
                        feas_sol[idx] = math.ceil(sol[idx])
                    else:
                        feas_sol[idx] = math.floor(sol[idx])
                else:
                    feas_sol[idx] = math.floor(sol[idx])
            else:
                feas_sol[idx] = math.ceil(sol[idx])
        feas_obj = np.sum(feas_sol * self.obj_coef)
        ## 以下判断用于debug
        if not self.check_sol_feas(feas_sol):
            print('simple rounding error!')
            return False,None,MAX
        return True,feas_sol,feas_obj

    def simple_rounding(self,solver,node):
        sol,sol_int_infeas = node.sol,node.sol_int_infeas
        return self._simple_rounding_internal(sol,sol_int_infeas)
    
    def int_rounding(self,solver,node):
        '''
        基于连续变量relaxation的rounding
        '''
        ## 引入控制流程决定是否执行，以平衡总体计算开销
        if node.level % 10 != 0:
            return False,None,MAX
        sample_prob = random.random()
        if self.rounding_count > 0 and sample_prob > self.rounding_threshold:
            return False,None,MAX
        self.rounding_count += 1
        
        ## 首先判断连续部分relax后，整数部分是否可以通过rounding得到可行解
        int_vars_bool = self.int_vars_bool
        sol,sol_int_infeas = node.sol,node.sol_int_infeas
        feas_sol = sol.copy()
        rows_val_int = self.A_csr.dot(feas_sol * int_vars_bool)
        infeas_flag = int_round_jit(int_vars_bool,feas_sol,sol_int_infeas,
                                rows_val_int,self.rows_lb_int,self.rows_ub_int,
                                self.down_locks_int,self.up_locks_int,self.max_locks_int,self.min_locks_int,
                                self.A_csr.indptr,self.A_csr.indices,self.A_csr.data,
                                self.A_csc.indptr,self.A_csc.indices,self.A_csc.data)
        
        if not infeas_flag:
            ## 如果整数部分rounding得到了可行解，用其固定变量上下界，再调用LP求解
            vars_lb_curr,vars_ub_curr = self.vars_lb.copy(),self.vars_ub.copy()
            self.vars_lb[int_vars_bool] = feas_sol[int_vars_bool]
            self.vars_ub[int_vars_bool] = feas_sol[int_vars_bool]
            self.problem.dual()
            self.vars_lb[int_vars_bool] = vars_lb_curr[int_vars_bool]
            self.vars_ub[int_vars_bool] = vars_ub_curr[int_vars_bool]
            if self.problem.getStatusCode() == 0:
                ## 求解LP找到了原问题的可行解
                feas_sol = self.primal_sol.copy()
                feas_obj = self.problem.objectiveValue
                return True,feas_sol,feas_obj
        return False,None,MAX
    
    def diving(self,solver,node):
        '''
        diving
        '''
        ## 引入控制流程决定是否执行，以平衡总体计算开销
        if node.level % 10 != 0:
            return False,None,MAX
        sample_prob = random.random()
        if self.diving_count > 0 and sample_prob > self.diving_threshold:
            return False,None,MAX
        self.diving_count += 1
        
        ## 恢复变量上下界
        solver.reset_problem_bounds()
        solver.modify_problem_bounds(node)
        sol = self.primal_sol
        feas_sol_flag,feas_sol,feas_obj = False,None,MAX
        delta_infeas = np.abs(sol - np.round(sol))
        sol_int_infeas = node.sol_int_infeas
        problem = self.problem
        for _ in range(self.dive_num):
            ## 做固定次数的dive
            if self.dive_type == HeuristicDiveType.INF:
                ## 根据不可行程度选择dive变量
                idx = sol_int_infeas[np.argmin(delta_infeas[sol_int_infeas])]
                val = sol[idx]
                val_new = np.round(val)
                if val_new < val:
                    self.vars_ub[idx] = val_new
                else:
                    self.vars_lb[idx] = val_new
            elif self.dive_type == HeuristicDiveType.COEF:
                ## 根据locks数选择dive变量
                min_locks_infeas = self.min_locks[sol_int_infeas]
                bool_valid = min_locks_infeas > 0
                idxs_valid = sol_int_infeas[bool_valid]
                min_locks = np.min(min_locks_infeas[bool_valid])
                idxs_min = idxs_valid[min_locks_infeas[bool_valid] == min_locks]
                if len(idxs_min) > 1:
                    idx = idxs_min[np.argmin(delta_infeas[idxs_min])]
                else:
                    idx = idxs_min[0]
                val = sol[idx]
                if self.down_locks[idx] < self.up_locks[idx]:
                    self.vars_ub[idx] = math.floor(val)
                elif self.down_locks[idx] > self.up_locks[idx]:
                    self.vars_lb[idx] = math.ceil(val)
                else:
                    val_new = np.round(val)
                    if val_new < val:
                        self.vars_ub[idx] = val_new
                    else:
                        self.vars_lb[idx] = val_new
                
            ## 选好变量并更新上下界后，重新求解LP 
            problem.dual()
            if problem.getStatusCode() != 0:
                ## LP不可行，abort
                feas_sol_flag = False
                break
                
            inf_cols = self.check_sol_int_feas(sol)
            if len(inf_cols) == 0:
                ## 找到整数可行解，返回结果
                feas_sol_flag,feas_sol,feas_obj = True,sol.copy(),problem.objectiveValue
                break
                
            feas_sol_flag,feas_sol,feas_obj = self._simple_rounding_internal(sol,inf_cols)
            if feas_sol_flag:
                ## simple rounding找到可行解，返回结果
                break
        
        ## 恢复变量上下界
        solver.reset_problem_bounds()
        solver.modify_problem_bounds(node)
        ## 以下判断用于debug
        if feas_sol_flag:
            if not self.check_sol_feas(feas_sol):
                print('diving error!')
                return False,None,MAX
        return feas_sol_flag,feas_sol,feas_obj
    
#############################
## 下面是numba实现jit加速的部分
from numba import njit

@njit
def get_inf_idxs_jit(rows_val,rows_lb,rows_ub,int_vars_bool,sol,inf_rows_base,inf_cols_base):
    ## 对给定的解sol和相应的行取值rows_val = A * sol，找到整数不可行的变量和约束不可行的行
    inf_rows,inf_cols = list(),list()
    for row in inf_rows_base:
        if (rows_val[row] < rows_lb[row] - TOL) or (rows_val[row] > rows_ub[row] + TOL):
            inf_rows.append(row)
    for col in inf_cols_base:
        if int_vars_bool[col] and abs(round(sol[col]) - sol[col]) > TOL:
            inf_cols.append(col)
    inf_rows,inf_cols = np.array(inf_rows,dtype=np.int32),np.array(inf_cols,dtype=np.int32)
    return inf_rows,inf_cols
    
@njit
def select_col_row_inf_jit(inf_rows,sol,int_vars_bool,
                            rows_val,rows_lb,rows_ub,
                            csr_indptr,csr_indices,csr_data):
    ## 根据对约束不可行程度的缓解情况和locks选rounding变量/列
    row = np.random.choice(inf_rows)
    row_val_,row_lb_,row_ub_ = rows_val[row],rows_lb[row],rows_ub[row]
    for mat_idx in range(csr_indptr[row],csr_indptr[row+1]):
        col,aij = csr_indices[mat_idx],csr_data[mat_idx]
        if int_vars_bool[col] and abs(round(sol[col]) - sol[col]) > TOL:
            ## 确保选出的变量目前是整数不可行的
            val = sol[col]
            val_up,val_down = math.ceil(val),math.floor(val)
            ## 向缓解约束不可行的方向round
            if row_val_ < row_lb_:
                if aij > 0:
                    val_new = val_up
                else:
                    val_new = val_down
                return col,val,val_new
            if row_val_ > row_ub_:
                if aij < 0:
                    val_new = val_up
                else:
                    val_new = val_down
                return col,val,val_new
    return -1,0,0
    
@njit
def select_col_int_inf_jit(inf_cols,sol,
                            rows_val,rows_lb,rows_ub,
                            down_locks,up_locks,max_locks,min_locks,
                            csc_indptr,csc_indices,csc_data):
    ## 根据locks和调整后不可行程度选rounding变量/列
    
    ## 直接选取max locks最大的变量进行处理
    col = inf_cols[np.argmax(max_locks[inf_cols])]
    val = sol[col]
    val_up,val_down = math.ceil(val),math.floor(val)
    ## 先根据locks数判断round方向: 选择locks小的方向
    if down_locks[col] < up_locks[col]:
        val_new = val_down
    elif down_locks[col] > up_locks[col]:
        val_new = val_up
    else:
        ## 如果两个方向locks数相同，则根据round后不可行程度的大小进行选择
        val_delta_down,val_delta_up = val_down - val,val_up - val
        row_inf_down_cnt,row_inf_up_cnt = 0,0
        row_inf_down_val,row_inf_up_val = 0,0
        for mat_idx in range(csc_indptr[col],csc_indptr[col+1]):
            row,aij = csc_indices[mat_idx],csc_data[mat_idx]
            row_val_,row_lb_,row_ub_ = rows_val[row],rows_lb[row],rows_ub[row]
            
            row_val_down = row_val_ + aij * val_delta_down
            if row_val_down < row_lb_ - TOL:
                row_inf_down_cnt += 1
                row_inf_down_val += row_lb_ - row_val_down
            if row_val_down > row_ub_ + TOL:
                row_inf_down_cnt += 1
                row_inf_down_val += row_val_down - row_ub_
            
            row_val_up = row_val_ + aij * val_delta_up
            if row_val_up < row_lb_ - TOL:
                row_inf_up_cnt += 1
                row_inf_up_val += row_lb_ - row_val_up
            if row_val_up > row_ub_ + TOL:
                row_inf_up_cnt += 1
                row_inf_up_val += row_val_up - row_ub_
            
        ## 先根据约束不可行数量选择方向
        if row_inf_down_cnt < row_inf_up_cnt:
            val_new = val_down
        elif row_inf_down_cnt > row_inf_up_cnt:
            val_new = val_up
        else:
            ## 再根据约束不可行程度选择方向
            if row_inf_down_val < row_inf_up_val:
                val_new = val_down
            elif row_inf_down_val > row_inf_up_val:
                val_new = val_up
            else:
                ## 如果仍然无法挑出方向，则选择固定方向或随机；这里选择向上round
                val_new = val_up
            
    return col,val,val_new

@njit
def int_round_jit(int_vars_bool,sol,inf_cols_base,rows_val_int,rows_lb_int,rows_ub_int,
                  down_locks_int,up_locks_int,max_locks_int,min_locks_int,
                  csr_indptr,csr_indices,csr_data,csc_indptr,csc_indices,csc_data):
    '''
    连续变量relax下的rounding尝试
    
    基本逻辑：
    1. 是否存在行或者列的不可行情况？(调用get_inf_idxs_jit)
        yes: goto 2
        no: 问题可行, 返回可行解
    2. 是否存在行不可行？
        yes: 随机挑选一个不可行的行，然后根据对不可行程度的缓解情况和locks选rounding变量/列, goto 3 (调用select_col_row_inf_jit)
        no: 根据locks和调整后不可行程度选rounding变量/列, goto 3 (调用select_col_int_inf_jit)
    3. 选出的变量/列是否有效？
        yes, 更新结果, goto 1
        no: 问题不可行, 返回failure
    '''
    infeas_flag = False
    num_row = len(rows_val_int)
    ## step 1: 检查行列的不可行性
    inf_rows,inf_cols = get_inf_idxs_jit(rows_val_int,rows_lb_int,rows_ub_int,
                                         int_vars_bool,sol,range(num_row),inf_cols_base)
    while (len(inf_rows) > 0 or len(inf_cols) > 0):
        ## step 2: 根据是否存在行不可行性，使用相应方法选出一个变量/列做rounding
        if len(inf_rows) > 0:
            col,val,val_new = select_col_row_inf_jit(inf_rows,sol,int_vars_bool,
                                        rows_val_int,rows_lb_int,rows_ub_int,
                                        csr_indptr,csr_indices,csr_data)
        else:
            col,val,val_new = select_col_int_inf_jit(inf_cols,sol,
                                        rows_val_int,rows_lb_int,rows_ub_int,
                                        down_locks_int,up_locks_int,max_locks_int,min_locks_int,
                                        csc_indptr,csc_indices,csc_data)
        ## step 3: 判断选出变量的有效性，并进行rounding
        if col >= 0:
            val_delta = val_new - val
            sol[col] = val_new
            for mat_idx in range(csc_indptr[col],csc_indptr[col+1]):
                rows_val_int[csc_indices[mat_idx]] += csc_data[mat_idx] * val_delta
        else:
            infeas_flag = True
            break
        ## step 1: 检查行列的不可行性
        inf_rows,inf_cols = get_inf_idxs_jit(rows_val_int,rows_lb_int,rows_ub_int,
                                         int_vars_bool,sol,range(num_row),inf_cols)
    return infeas_flag

下面，我们沿用已有的求解入口来对上述heuristic方法进行测试。

In [5]:
from util import *

def bb_solve(model_name,branch_type,select_type,propagate_type,time_limit=3600,if_disp=True):
    ## 读取问题
    s = CyClpSimplex()
    s.readMps(model2fname(model_name))
    s.logLevel = 0
    solver = BBSolverExtend(s,branch_type,select_type,propagate_type)
    dt,status,LB,obj = solver.solve(time_limit=time_limit,if_disp=if_disp,disp_iters=100000)
    total_node_cnt,remain_node_cnt = len(solver.nodes),len(solver.unprocessed_node_idxs)
    for node in solver.nodes:
        node.clear()
    return dt,status,LB,obj,total_node_cnt,remain_node_cnt,solver

## 读取测试案例
with open('data/test_cases_v3','r+') as f:
    lines = f.readlines()
model_names = [line.strip('\n') for line in lines]

In [7]:
#############################
branch_type = BranchType.PSEUDO
select_type = SelectType.BE_PLUNGE
propagate_type = PropagateType.LINEAR_OBJ
#############################

max_secs = 600 ## 每个问题限时10分钟
result = dict()
for model_name in model_names:
    dt,status,LB,obj,total_node_cnt,remain_node_cnt,_ = \
        bb_solve(model_name,branch_type,select_type,propagate_type,time_limit=max_secs,if_disp=False)
    print('{}: time={:.2f}, status={}. LB={:.3e}, obj={:.3e}. total nodes={}, remain nodes={}.'.format(
        model_name,dt,status,LB,obj,total_node_cnt,remain_node_cnt))
    result[model_name] = (dt,status,LB,obj,total_node_cnt,remain_node_cnt)

## 保存测试结果
## 对于不同的branching/node selection/propagate/heuristic配置，记得修改保存的文件名，以免覆盖原来的结果
df_result = process_result_new(result,save_fname='result/ours_benchmark_v4_psc_bep.csv')
print('elapsed hours: {:.2f}.'.format(df_result['time'].sum() / 3600))

binkar10_1: time=600.00, status=SolveStatus.ONGOING. LB=6.643e+03, obj=6.757e+03. total nodes=298243, remain nodes=74351.
dano3_3: time=92.21, status=SolveStatus.OPT. LB=5.763e+02, obj=5.763e+02. total nodes=91, remain nodes=0.
dano3_5: time=605.08, status=SolveStatus.ONGOING. LB=5.762e+02, obj=5.771e+02. total nodes=641, remain nodes=112.
eil33-2: time=92.42, status=SolveStatus.OPT. LB=9.340e+02, obj=9.340e+02. total nodes=11209, remain nodes=0.
gen-ip002: time=437.29, status=SolveStatus.ONGOING. LB=-4.832e+03, obj=-4.784e+03. total nodes=1274375, remain nodes=274374.
gen-ip054: time=441.04, status=SolveStatus.ONGOING. LB=6.771e+03, obj=6.855e+03. total nodes=1387687, remain nodes=387686.
gmu-35-40: time=600.00, status=SolveStatus.ONGOING. LB=-2.407e+06, obj=-2.401e+06. total nodes=875437, remain nodes=375103.
gmu-35-50: time=600.00, status=SolveStatus.ONGOING. LB=-2.608e+06, obj=-2.570e+06. total nodes=595497, remain nodes=258171.
icir97_tension: time=600.00, status=SolveStatus.ONGOI

通过上述测试，我们可以得到以下结论：

* 引入primal heuristics会明显降低计算效率（同样时间限制内遍历更少结点）。
* 引入primal heuristics寻找可行解的效率有一定提升（在更短的时间内找到可行解），但是对于较难的问题并不能保证在有限时间内找到可行解，也不能保证在同样有限的时间内比不使用primal heuristics找到更好的可行解。
* primal heuristics最好应该结合对实际问题的理解进行设计：在MIP建模过程中，问题的结构化信息会有所损失，general heuristics很难保证较好的效果。例如，在测试问题neos-3627168-kasai中，如果我们将int_rounding的tie-breaking rounding方向从向上改为向下，则上述流程将很难找到可行解。而而“向上round更有利于寻找可行解”这一结论是可以通过分析具体问题形式得出的。